In [154]:
import pandas as pd
import json

In [155]:
df_corpus = pd.read_csv("corpus.tsv", delimiter='\t')

In [156]:
df_features = pd.read_csv("node_features.csv")

In [157]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    return data

In [158]:
splits_file = "./data/emnlp2018/splits.json"
splits = load_json(splits_file)

In [159]:
bias_mapping = {
    "left": "left", 'extreme-left': "left",
    "center": "center", 'right-center': "center", 'left-center': "center",
    "right": "right", 'extreme-right': "right"
}

# fact_mapping = {"low": 0, "mixed": 1, "high": 2}

url_mapping = {
    "conservativeoutfitters.com": "conservativeoutfitters.com-blogs-news",
    "who.int": "who.int-en",
    "themaven.net": "themaven.net-beingliberal",
    "al-monitor.com": "al-monitor.com-pulse-home.html",
    "pri.org": "pri.org-programs-globalpost",
    "mlive.com": "mlive.com-grand-rapids-#-0",
    "pacificresearch.org": "pacificresearch.org-home",
    "telesurtv.net": "telesurtv.net-english",
    "elpais.com": "elpais.com-elpais-inenglish.html",
    "inquisitr.com": "inquisitr.com-news",
    "cato.org": "cato.org-regulation",
    "jpost.com": "jpost.com-Jerusalem-Report",
    "newcenturytimes.com": "newcenturytimes.com",
    "oregonlive.com": "oregonlive.com-#-0",
    "rfa.org": "rfa.org-english",
    "people.com": "people.com-politics",
    "russia-insider.com": "russia-insider.com-en",
    "nola.com": "nola.com-#-0",
    "host.madison.com": "host.madison.com-wsj",
    "conservapedia.com": "conservapedia.com-Main_Page",
    "futureinamerica.com": "futureinamerica.com-news",
    "indymedia.org": "indymedia.org-or-index.shtml",
    "newyorker.com": "newyorker.com-humor-borowitz-report",
    "rt.com": "rt.com-news",
    "westernjournalism.com": "westernjournalism.com-thepoint",
    "scripps.ucsd.edu": "scripps.ucsd.edu-news",
    "citizensunited.org": "citizensunited.org-index.aspx",
    "gallup.com": "gallup.com-home.aspx",
    "news.harvard.edu": "news.harvard.edu-gazette",
    "spin.com": "spin.com-death-and-taxes",
    "itv.com": "itv.com-news",
    "theguardian.com": "theguardian.com-observer",
    "concernedwomen.org": "concernedwomen.org-blog",
    "npr.org": "npr.org-sections-news",
    "yahoo.com": "yahoo.com-news-?ref=gs",
    "zcomm.org": "zcomm.org-zmag",
    "therealnews.com": "therealnews.com-t2"
}
df_features = df_features.replace({"source_url_normalized": url_mapping})
df_corpus = df_corpus.replace({"bias": bias_mapping})
#df_corpus = df_corpus.replace({"fact": fact_mapping})

In [160]:
output1 = pd.merge(df_features, df_corpus, 
                   on='source_url_normalized', 
                   how='right')

In [161]:
column_means = output1.mean()
output1 = output1.fillna(column_means)


/tmp/ipykernel_188982/3579042041.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  column_means = output1.mean()


In [162]:
output1 = output1.drop(['source_url', 'ref', 'alexa_ranks', 'daily_pageviews_per_visitors'], axis = 1)

In [166]:
from typing import Dict, List
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
def calculate_metrics(actual, predicted):
    """
    Calculate performance metrics given the actual and predicted labels.
    Returns the macro-F1 score, the accuracy, the flip error rate and the
    mean absolute error (MAE).
    The flip error rate is the percentage where an instance was predicted
    as the opposite label (i.e., left-vs-right or high-vs-low).
    """
    # calculate macro-f1
    f1 = f1_score(actual, predicted, average='macro') * 100

    # calculate accuracy
    accuracy = accuracy_score(actual, predicted) * 100
    return f1, accuracy

def train_model(splits: Dict[str, Dict[str, List[str]]],
                features: Dict[str, Dict[str, List[float]]], task):
    
    
    all_urls = []
    actual = []
    predicted = []
    
    if task=="fact":
        other_task = "bias"
    
    if task=="bias":
        other_task = "fact"
    
    i = 0
    num_folds = len(splits)
    for f in range(num_folds):
        # get the training and testing media for the current fold
        urls = {
            "train": splits[str(f)]["train"],
            "test": splits[str(f)]["test"],
        }

        all_urls.extend(splits[str(f)]["test"])

        
        # concatenate the different features/labels for the training sources
        X_train = features[features["source_url_normalized"].isin(urls["train"])]
        X_train = X_train.drop(['source_url_normalized', task, other_task], axis = 1)
        #print(X_train.head())
        y_train = np.asarray(features[features["source_url_normalized"].isin(urls["train"])][task])
        #print(y_train)
        
        X_test = features[features["source_url_normalized"].isin(urls["test"])]
        X_test = X_test.drop(['source_url_normalized', task, other_task], axis = 1)
        y_test = np.asarray(features[features["source_url_normalized"].isin(urls["test"])][task])
        
        clf = LabelPropagation()

        # train the classifier using the training data
        clf.fit(X_train, y_train)
        print(clf.score(X_test, y_test))

        # generate predictions
        pred = clf.predict(X_test)

        # generate probabilites
        prob = clf.predict_proba(X_test)
        #print(y_test)
        # cumulate the actual and predicted labels, and the probabilities over the different folds.  then, move the index
        actual[i: i + y_test.shape[0]] = y_test
        predicted[i: i + y_test.shape[0]] = pred
        i += y_test.shape[0]
        

    # calculate the performance metrics on the whole set of predictions (5 folds all together)
    f1, accuracy = calculate_metrics(actual, predicted)
    print(f1, accuracy)
    

In [167]:
train_model(splits, output1, "fact")

0.5767441860465117
0.5186915887850467
0.5539906103286385
0.5660377358490566
0.6084905660377359
41.94512124376378 56.47279549718574


/home/utsav/.pyenv/versions/nmp/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/home/utsav/.pyenv/versions/nmp/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/home/utsav/.pyenv/versions/nmp/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/home/utsav/.pyenv/versions/nmp/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/home/utsav/.pyenv/versions/nmp/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/home/utsa

In [57]:
labels

array(['high', 'high', 'low', ..., 'high', 'high', 'mixed'], dtype=object)

In [58]:
features = output1.drop(['fact', 'source_url_normalized'], axis=1)

In [59]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, 
    test_size=0.20)

In [60]:
from sklearn.semi_supervised import LabelPropagation

# initialize
clf = LabelPropagation()

# train the classifier using the training data
clf.fit(features_train, labels_train)

LabelPropagation()

In [61]:
# compute accuracy using test data
acc_test = clf.score(features_test, labels_test)

print ("Test Accuracy:", acc_test)
# Test Accuracy: 0.98

Test Accuracy: 0.602803738317757


/home/utsav/.pyenv/versions/nmp/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
